In [107]:
import pandas as pd
from pathlib import Path
import numpy as np
import hvplot.pandas
import scipy.optimize
import symbol_data
import portfolio_data

In [67]:
spx_df = pd.read_csv(Path('./Resources/SPX.csv'), 
                    index_col='Date', 
                    parse_dates=True,
                    infer_datetime_format=True
                    )
spx_daily_returns = spx_df['Close'].pct_change().dropna()

In [68]:
stocks = ['AAPL', 'ACN', 'ADBE', 'ADI', 'ADSK', 'AKAM', 'AMAT', 'AMD', 'CRM', 'DELL', 'DOCU', 'EPAM', 'FIS', 'FISV', 'IBM', 'INFY', 'LRCX', 'MSFT', 'MU', 'NOW', 'NVDA', 'ORCL', 'SAP', 'TEAM', 'TXN', 'WDAY', 'WORK', 'XLNX', 'ZBRA', 'ZEN', 'ZI', 'ZS']
# This function may have to be rewritten to accomodate Anurag's data functions
# as it performs all the analysis within the same loop as retrieving the data.
def stock_data_calculator(stocks):
    # Create an empty dictionary to store the stock data
    stock_data_dict = {}
    # This section can be replaced by Anurag's function
    for stock in stocks:
        # stock_df = pd.read_csv(
        #     Path(f'./Resources/{stock}.csv'), 
        #     index_col = 'Date', 
        #     parse_dates = True, 
        #     infer_datetime_format=True
        #     )
        stock_df = portfolio_data.get_portfolio_historical_data(stock)
        daily_returns = stock_df['Close'].pct_change().dropna()
        daily_returns_df = pd.concat(
            [daily_returns, spx_daily_returns], 
            axis=1,
            join='inner', 
            keys = [stock, 'SPX']
            )
        covariance = daily_returns_df[stock].cov(daily_returns_df['SPX'])
        beta = covariance / daily_returns_df['SPX'].var()

        std = daily_returns.std()

        var = daily_returns.var()

        expected_return = .035 + beta*(.1-.035)
        
        stock_data_dict[stock] = {'expected_return': expected_return, 'std': std, 'var': var, 'beta': beta}
    return stock_data_dict

        


In [69]:
stock_data_calculator(stocks)

{'AAPL': {'expected_return': 0.1021857914988607,
  'std': 0.02079129793240788,
  'var': 0.0004322780697141482,
  'beta': 1.0336275615209338},
 'ACN': {'expected_return': 0.0928258962563413,
  'std': 0.01645928931954835,
  'var': 0.0002709082049045985,
  'beta': 0.8896291731744815},
 'ADBE': {'expected_return': 0.11004981725358733,
  'std': 0.021328535684571374,
  'var': 0.00045490643444803455,
  'beta': 1.1546125731321126},
 'ADI': {'expected_return': 0.10267257755225005,
  'std': 0.01951063375075435,
  'var': 0.0003806648293560747,
  'beta': 1.0411165777269238},
 'ADSK': {'expected_return': 0.11739155821007816,
  'std': 0.024441902649579843,
  'var': 0.0005974066051315382,
  'beta': 1.2675624340012024},
 'AKAM': {'expected_return': 0.1100147458956259,
  'std': 0.027881047884079174,
  'var': 0.0007773528311143157,
  'beta': 1.15407301377886},
 'AMAT': {'expected_return': 0.11417312976324465,
  'std': 0.022677234403613647,
  'var': 0.0005142569601964385,
  'beta': 1.2180481502037637},
 

In [70]:
def portfolio_expected_return_calculator(stocks, weights):
    # Create an empty list which will hold the expected returns
    expected_returns = []
    # Gather the stock data from the symbol_data_calculator function
    symbol_data = stock_data_calculator(stocks)
    # Pull only the expected returns from the data
    for symbol in symbol_data:
        expected_returns.append(symbol_data[symbol]['expected_return'])
    # Multiply the expected returns by the weights to get weighted expected returns
    weighted_expected_returns = expected_returns * weights
    # Sum all these to get the expected return of the portfolio
    weighted_expected_returns = weighted_expected_returns.sum()
    return weighted_expected_returns 
    
   

In [71]:
weights = np.random.random(32)
weights /= weights.sum()
portfolio_expected_return_calculator(stocks, weights)

0.10524449746872866

In [72]:
def portfolio_variance_calculator(symbols, weights):
    # Getting the data. This part can be replaced by anurag's function
    df_dict = {}
    for symbol in symbols:
        symbol_df = pd.read_csv(
            Path(f'./Resources/{symbol}.csv'),
            index_col = 'Date',
            parse_dates = True,
            infer_datetime_format = True
            )
        df_dict[symbol] = symbol_df['Close']

    prices_df = pd.concat(df_dict.values(), axis = 1, join = 'inner', keys = stocks)
    daily_returns = prices_df.pct_change().dropna()
    
    # Get the covariance array of the portfolio using the .cov() method.
    portfolio_cov = np.array(daily_returns.cov())

    # Get the standard deviation of each asset in the portfolio using the .std() method.
    portfolio_std = np.array(daily_returns.std())

    # Cross multiply the std array with the transposition of itself.
    portfolio_stdT = np.transpose(np.array([portfolio_std]))
    portfolio_std_cp = portfolio_std * portfolio_stdT

    # Get the correlation matrix by dividing the covariance matrix by the standard deviation matrix.
    correlation_matrix = portfolio_cov / portfolio_std_cp

    # Get weighted standard deviation and save the transposition of that array.
    weighted_std = portfolio_std * weights
    weighted_stdT = np.transpose(np.array([weighted_std]))

    # Calculate portfolio variance by first multiplying the covariance matrix by the 
    # weighted standard deviation array. This will give an array which is 
    # 1x<the number of stocks in the portfolio>
    portfolio_var = np.matmul(weighted_std, correlation_matrix)
    # Then multiplying that array by the transposition of the weighted standard deviation array.
    # This will give a single value.
    portfolio_var = np.matmul(portfolio_var, weighted_stdT)
    # And finally taking the square root of that value.
    portfolio_var = np.sqrt(portfolio_var)
    return portfolio_var




In [73]:
weights = np.random.random(32)
weights /= weights.sum()
portfolio_variance_calculator(stocks, weights)


array([0.01700711])

In [74]:
def portfolio_performance_calculator(stocks, weights):
    # This section can be replaced by Anurag's function
    df_dict = {}
    for stock in stocks:
        stock_df = pd.read_csv(
            Path(f'./Resources/{stock}.csv'),
            index_col = 'Date',
            parse_dates = True,
            infer_datetime_format = True
            )
        stock_df['daily_returns'] = stock_df['Close'].pct_change().dropna()
        df_dict[stock] = stock_df[['daily_returns']]

    stocks_df = pd.concat(df_dict.values(), axis = 1, join = 'inner', keys = stocks)

    # Calculate cumulative returns for each stock in the portfolio.
    cumulative_returns = (1+stocks_df).cumprod() - 1
    # Multiply each stock by its respective weight.
    cumulative_returns = cumulative_returns * weights
    # Sum all the weighted cumulative returns
    cumulative_returns = cumulative_returns.sum(axis=1)
    return cumulative_returns

In [75]:
# This function accepts a list of stocks and a list of weights which must be in the 
# same order. It will return the 95% confidence interval for annual performance
# of the portfolio.
def portfolio_95percent_confidence_calculator(stocks, weights):
    expected_return = portfolio_expected_return_calculator(stocks, weights)
    variance = portfolio_variance_calculator(stocks, weights)
    lower_bound = expected_return - 2*variance
    upper_bound = expected_return + 2*variance
    return lower_bound, upper_bound

In [76]:
(stocks, weights)

(['AAPL',
  'ACN',
  'ADBE',
  'ADI',
  'ADSK',
  'AKAM',
  'AMAT',
  'AMD',
  'CRM',
  'DELL',
  'DOCU',
  'EPAM',
  'FIS',
  'FISV',
  'IBM',
  'INFY',
  'LRCX',
  'MSFT',
  'MU',
  'NOW',
  'NVDA',
  'ORCL',
  'SAP',
  'TEAM',
  'TXN',
  'WDAY',
  'WORK',
  'XLNX',
  'ZBRA',
  'ZEN',
  'ZI',
  'ZS'],
 array([0.00796418, 0.00970689, 0.00430216, 0.03814624, 0.03512294,
        0.03962741, 0.04132653, 0.0423957 , 0.02582287, 0.00261783,
        0.04008024, 0.05341208, 0.01865945, 0.01817888, 0.05211889,
        0.03449983, 0.05917331, 0.01277487, 0.00835608, 0.02860374,
        0.02866001, 0.05699303, 0.03078902, 0.01907603, 0.00884182,
        0.05689261, 0.04854696, 0.01578096, 0.02045994, 0.05989768,
        0.05880154, 0.02237026]))

In [77]:
def efficient_frontier_generator(stocks):
    i = 0
    portfolio_dict = {}
    while i < 5:
        weights = np.random.random(32)
        weights /= weights.sum()
        portfolio_return = portfolio_expected_return_calculator(stocks, weights)
        portfolio_variance = portfolio_variance_calculator(stocks, weights)
        portfolio_dict[i] = (portfolio_return, portfolio_variance, weights)
        i += 1
    return portfolio_dict



In [78]:
# Takes a naive approach to finding the portfolio with the minimum risk.
# There is a way to minimize a function with SciPy.optimize.minimize but 
# I can't figure out how to do it. 
# This function takes a list of stocks and creates 100 random portfolios
# and returns the one with the minimum risk.
def naive_minimum_risk_finder(stocks):
    # Create an iterator
    i = 0
    # Create dictionary to store the minimum risk portfolio
    min_risk = {}
    # Generate 100 portfolios with random weightings.
    # Save only the one with the minimim risk.
    while i < 100:
        weights = np.random.random(len(stocks))
        weights /= weights.sum()
        risk = portfolio_variance_calculator(stocks, weights)
        if not min_risk:
            min_risk['min_risk']={'risk':risk, 'weights':weights}
        elif risk < min_risk['min_risk']['risk']:
            min_risk['min_risk']={'risk':risk, 'weights':weights}
        i += 1
    return min_risk


In [79]:
naive_minimum_risk_finder(stocks)

{'min_risk': {'risk': array([0.01557095]),
  'weights': array([0.03555009, 0.04106247, 0.02711876, 0.05355551, 0.05085075,
         0.05569532, 0.02766137, 0.00417184, 0.04132095, 0.05444134,
         0.02167664, 0.00168093, 0.0427039 , 0.04541502, 0.04944397,
         0.04301324, 0.00714041, 0.00122269, 0.05418961, 0.00834471,
         0.01915928, 0.01707678, 0.03105469, 0.05607927, 0.02102854,
         0.00626393, 0.05665821, 0.03233294, 0.03812038, 0.01266993,
         0.01872183, 0.0245747 ])}}

In [80]:
# Takes a naive approach to finding the portfolio with the minimum risk.
# There is a way to minimize a function with SciPy.optimize.minimize but 
# I can't figure out how to do it. 
# This function takes a list of stocks and creates 100 random portfolios
# and returns the one with the maximum return.
def naive_maximum_return_finder(stocks):
    i = 0
    # Create dictionary to store the maximum return portfolio
    max_return = {}
    # Generate 100 portfolios with random weightings.
    # Save only the one with the minimim risk.
    while i < 100:
        weights = np.random.random(len(stocks))
        weights /= weights.sum()
        returns = portfolio_expected_return_calculator(stocks, weights)
        if not max_return:
            max_return['max_return']={'return':returns, 'weights':weights}
        elif returns > max_return['max_return']['return']:
            max_return['max_return']={'return':returns, 'weights':weights}
        i += 1
    return max_return

In [81]:
naive_maximum_return_finder(stocks)

{'max_return': {'return': 0.10868263191557352,
  'weights': array([0.01969949, 0.02750897, 0.00905616, 0.00824853, 0.06298531,
         0.04530381, 0.00708762, 0.05963186, 0.01863723, 0.0325562 ,
         0.00560353, 0.04303172, 0.00824645, 0.01585717, 0.05639183,
         0.03405789, 0.03768874, 0.02112605, 0.04026231, 0.0279426 ,
         0.05565747, 0.03549286, 0.00933088, 0.02077517, 0.06124776,
         0.043136  , 0.02282959, 0.05938845, 0.00284033, 0.05762949,
         0.03573454, 0.015014  ])}}

In [108]:
symbol_data.get_historical_data('SPX')

,Date,Open,High,Low,Close,Adj Close,Volume
0,1927-12-30,17.660000,17.660000,17.660000,17.660000,17.660000,0
1,1928-01-03,17.760000,17.760000,17.760000,17.760000,17.760000,0
2,1928-01-04,17.719999,17.719999,17.719999,17.719999,17.719999,0
3,1928-01-05,17.549999,17.549999,17.549999,17.549999,17.549999,0
4,1928-01-06,17.660000,17.660000,17.660000,17.660000,17.660000,0
...,...,...,...,...,...,...,...
23318,2020-10-29,3277.169922,3341.050049,3259.820068,3310.110107,3310.110107,4903070000
23319,2020-10-30,3293.590088,3304.929932,3233.939941,3269.959961,3269.959961,4840450000
23320,2020-11-02,3296.199951,3330.139893,3279.739990,3310.239990,3310.239990,4310590000
23321,2020-11-03,3336.250000,3389.489990,3336.250000,3369.159912,3369.159912,4220070000
